In [1]:
# -*- coding: utf-8 -*-
import argparse
import os
import string
import sys
os.chdir('/home/hyeryung_son/mucoco')
from itertools import repeat
import torch.multiprocessing as mp
from typing import List

import numpy as np
import pandas as pd
import torch
import transformers
from datasets import Dataset
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModelForMaskedLM, AutoTokenizer, AutoConfig

from new_module.utils.robertacustom import RobertaCustomForSequenceClassification
from new_module.locate.new_locate_utils import *
import new_module.losses as lossbuilder

import logging
import os
from typing import List

import torch
import torch.nn.functional as F
import transformers

In [2]:
## setup for prototyping
logging.basicConfig(level=logging.DEBUG, format="%(message)s")
logger = logging.getLogger(__name__)
logger.setLevel(os.environ.get("LOGGING_LEVEL", logging.DEBUG))

config={#'model_paths':['gpt2-large','/shared/s3/lab07/hyeryung/loc_edit/roberta-base-jigsaw-toxicity-classifier-energy-training/step_600_best_checkpoint'],
        # 'tokenizer_paths':['gpt2-large','/shared/s3/lab07/hyeryung/loc_edit/roberta-base-jigsaw-toxicity-classifier-energy-training/step_600_best_checkpoint'],
        'model_paths':['gpt2-large','models/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/checkpoint_best'],
        'tokenizer_paths':['gpt2-large','models/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/checkpoint_best'],
        'model_types': ["AutoModelForCausalLM", "AutoModelForSequenceClassification"],
        'cache_dir': "hf_cache",
        'target_type': "embeds",
        'method': "mlm-beamsearch-v0",
       'losses': ["gpt2", "classification_no_prefix_logprobloss"],
       'target_label_ids': [0,0] ,
       'build_loss_dict': {"coeff_steps": 200, "coeff_pattern": "constant", "loss_type": "xentropy", "length_normalize": False, "AR_temperature": 1.0, "AR_top_k": 0, "AR_top_p": 0.96, "max_output_length": 20},
       'min_epsilons': [0.75],
       'source_data': 'new_module/data/toxicity-avoidance/testset_gpt2_2500.jsonl',
       'locate_unit': 'word',
       'locate_method': 'grad_norm',
       'device': 'cuda',
       'k_per_location': 10,
       'closs_weight': 0.9,
       'beam_size': 3}

class dummyArgs:
    def __init__(self, **kwargs):
        for k, v in kwargs.items():
            setattr(self, k, v)

build_loss_args = dummyArgs(**config["build_loss_dict"])

In [3]:
name2tokenizer = {}
name2model = {}
name2config = {}
loss2tokenizer = {}
embed_luts = []
primary_model = None
for i, model_path in enumerate(config["model_paths"]):
    if (
        model_path not in name2model
    ):  # making sure we are not loading the model twice in case some constraints use the same model.
        try:
            name2tokenizer[model_path] = AutoTokenizer.from_pretrained(
                config["tokenizer_paths"][i],
                cache_dir=config["cache_dir"],
                use_fast=True,
            )
        except:
            name2tokenizer[model_path] = AutoTokenizer.from_pretrained(
                config["tokenizer_paths"][i],
                cache_dir=config["cache_dir"],
                use_fast=False,
            )

        name2config[model_path] = AutoConfig.from_pretrained(
            model_path, cache_dir=config["cache_dir"]
        )

        if "Custom" in config["model_types"][i]:
            name2model[model_path] = lossbuilder.ModelWrapper(
                getattr(utils, config["model_types"][i]).from_pretrained(
                    model_path,
                    config=name2config[model_path],
                    cache_dir=config["cache_dir"],
                )
            )
        else:
            name2model[model_path] = lossbuilder.ModelWrapper(
                getattr(transformers, config["model_types"][i]).from_pretrained(
                    model_path,
                    config=name2config[model_path],
                    cache_dir=config["cache_dir"],
                )
            )
        name2model[model_path].eval()
        name2model[model_path].cuda()

    input_embeds = name2model[model_path].get_input_embeddings()
    if isinstance(input_embeds, torch.nn.Sequential):
        input_embeds = input_embeds[0]
    embed_luts.append(input_embeds)

    if config["target_type"] == "embeds":
        embed_luts[-1].requires_grad = False

    if i == 0:
        primary_model = name2model[model_path]

mlm_tokenizer = AutoTokenizer.from_pretrained("roberta-base")
mlm = None if config["method"] == "mlm-beamsearch-v2" else AutoModelForMaskedLM.from_pretrained("roberta-base")  

lossfns = []
for i, loss in enumerate(config["losses"]):
    lossfns.append(
        lossbuilder.build_loss(
            loss,
            name2model[config["model_paths"][i]],
            name2tokenizer[config["model_paths"][i]],
            build_loss_args,
        )
    )
    lossfns[i].tokenizer.add_special_tokens({"mask_token": mlm_tokenizer.mask_token})
    loss2tokenizer[loss] = lossfns[i].tokenizer

primary_tokenizer = name2tokenizer['gpt2-large']
secondary_tokenizer = list(name2tokenizer.values())[1]

Starting new HTTPS connection (1): huggingface.co:443


https://huggingface.co:443 "HEAD /gpt2-large/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
https://huggingface.co:443 "HEAD /gpt2-large/resolve/main/config.json HTTP/1.1" 200 0
https://huggingface.co:443 "HEAD /gpt2-large/resolve/main/generation_config.json HTTP/1.1" 200 0
Some weights of the model checkpoint at models/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/checkpoint_best were not used when initializing RobertaForSequenceClassification: ['roberta.embeddings.word_embeddings.0.weight', 'roberta.embeddings.word_embeddings.1.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceC

In [17]:
def unravel(outputs_df):
    outputs_df=outputs_df.explode('generations',ignore_index=True)
    outputs_df['prompt']=outputs_df['prompt'].apply(lambda x: x['text'])
    outputs_df['generations']=outputs_df['generations'].apply(lambda x: x['text'] if isinstance(x, dict) else x)
    outputs_df = outputs_df.dropna().reset_index(drop=True)
    return outputs_df
toxic_gen=pd.read_json('new_module/data/toxicity-avoidance/dev_set.jsonl',lines=True)
toxic_gen=unravel(toxic_gen)
prompt = '“I went to use the bathroom, and I went in and Lance was bent'
prediction = toxic_gen.loc[toxic_gen['prompt']==prompt,'generations'].tolist()
ckpt_path = config['model_paths'][1]
model = name2model[ckpt_path] #AutoModelForSequenceClassification.from_pretrained(ckpt_path)
tokenizer = name2tokenizer[ckpt_path] #AutoTokenizer.from_pretrained(ckpt_path)
device='cuda'
model = model.to(device)
loc_machine=LocateMachine(model,tokenizer)
masked_text = loc_machine.locate_main(prediction, "grad_norm", max_num_tokens = 6, unit="word", num_layer=10, label_id=0)

In [18]:
mlm = mlm.to(device)

In [19]:
source_text=prompt

In [21]:
masked_text

["<mask> me trying to cum. I<mask> hear<mask><mask> to come, but I didn't stop him. Thank God he had<mask><mask> on me.",
 ' over like<mask> and he was<mask><mask>',
 ' over me and put his hand in my<mask> and my<mask> forced his hand up and down my ass and<mask> he put his hand back<mask> my<mask> and he rubbed it.This<mask>',
 '<mask><mask> reaching for his<mask><mask> and so,<mask><mask><mask><mask> the stall door open and I get<mask><mask> Stephanie said.',
 ' over me and he was raping me. And I<mask> away and he was fighting me, and I<mask><mask><mask><mask><mask> had no idea what it was<mask> to',
 ' over the sink. It was dirty. The whole<mask><mask><mask><mask> the sink and the tub.',
 '<mask><mask> sink and then was<mask> over one of the water<mask> and being a moron in his<mask><mask>',
 ' over the toilet. He<mask> kept all this time going<mask> a donkey trying to put a little<mask><mask><mask><mask><mask>',
 ' over and I flicked my underwear down to a certain<mask><mask><mask

In [22]:
## replace tokens at the indices with mask tokens
inputs = mlm_tokenizer(
    masked_text, return_tensors="pt", padding=True, truncation=True
)
inputs = inputs.to(device) 

## make predictions for the masked indices
with torch.no_grad():
    logits = mlm(**inputs).logits

special_token_ids = mlm_tokenizer.convert_tokens_to_ids(mlm_tokenizer.all_special_tokens)
logits[:, :, special_token_ids] = -float("inf")

indices_in_mlm_tokens = (
    inputs.input_ids == mlm_tokenizer.mask_token_id
).nonzero(as_tuple=True)

## get top k tokens for each index
predicted_token_ids = torch.topk(
    logits[indices_in_mlm_tokens[0], indices_in_mlm_tokens[1], :],
    k=config['k_per_location'],
    dim=-1,
)
# print(f"predicted_token_ids: {predicted_token_ids}")
# print(f"mlm_tokenizer.batch_decode(predicted_token_ids.indices): {mlm_tokenizer.batch_decode(predicted_token_ids.indices)}")


In [23]:
mlm_tokenizer = AutoTokenizer.from_pretrained("roberta-base")

Resetting dropped connection: huggingface.co
https://huggingface.co:443 "HEAD /roberta-base/resolve/main/tokenizer_config.json HTTP/1.1" 200 0


In [24]:
inputs = mlm_tokenizer(
    masked_text, return_tensors="pt", padding=True, truncation=True
)
inputs = inputs.to(device) 

In [25]:
input_ids=inputs['input_ids']

In [26]:
edit_indices = sorted(list(set(indices_in_mlm_tokens[1].tolist())))

In [27]:
i = 1
hypotheses = input_ids[:, None, :].repeat((1,config['beam_size'],1))

for i in edit_indices:
    
    batch_ids_to_edit = indices_in_mlm_tokens[0][indices_in_mlm_tokens[1]==i]

    tmp_hypotheses = hypotheses[batch_ids_to_edit].detach().clone()
    tmp_hypotheses=tmp_hypotheses.repeat((1,config['k_per_location'],1))
    expanded_tmp_hypotheses = tmp_hypotheses.clone()

    candidates = predicted_token_ids.indices[(indices_in_mlm_tokens[1]==i).nonzero().squeeze(-1),:]
    candidates = candidates[:, :, None].repeat((1,1, config['beam_size'])).reshape(candidates.shape[0], -1,1)

    tmp_hypotheses = torch.cat((tmp_hypotheses[:, :, :i], candidates),dim=-1) ## tmp_hypotheses: [(a,b,c),(a,b,c), ..., (a,b,c)], candidates: [(p,p,p), (q,q,q), ..., (v,v,v)]
    tmp_hypotheses = tmp_hypotheses.reshape(-1, tmp_hypotheses.shape[-1])

    loss_weights = [1 - config['closs_weight'], config['closs_weight']]
    curr_loss = torch.zeros(tmp_hypotheses.shape[0]).to(config['device'])
    for lossid, lossname in enumerate(config["losses"]):
        with torch.no_grad():
            lossvalue = lossfns[lossid].compute_gold_loss(
                source_text, mlm_tokenizer.batch_decode(tmp_hypotheses,skip_special_tokens=True),
                label_id=config['target_label_ids'][lossid],
            )
        torch.cuda.empty_cache()
        curr_loss += loss_weights[lossid] * lossvalue
        
    curr_loss = torch.stack(torch.split(curr_loss, config['beam_size'] * config['k_per_location'], dim=0),dim=0)
    top_beams=torch.topk(curr_loss, k=(config['beam_size']*2+1), dim=-1, largest=False).indices

    tmp_hypotheses = torch.split(tmp_hypotheses, config['beam_size'] * config['k_per_location'], dim=0) ## 아래의 작업을 더 간단히 할 수 있는 방법?
    tmp_hypotheses = torch.stack([x[top_beams[j]] for j, x in enumerate(tmp_hypotheses)],dim=0)
    tmp_hypotheses = torch.unique(tmp_hypotheses, dim=1)[:, :config['beam_size'], :]

    hypotheses[batch_ids_to_edit,:, i]=tmp_hypotheses[:, :, i]

torch.Size([4, 7, 2])
torch.Size([4, 3, 2])
torch.Size([2, 7, 3])
torch.Size([2, 3, 3])
torch.Size([1, 7, 4])
torch.Size([1, 3, 4])
torch.Size([2, 7, 7])
torch.Size([2, 3, 7])
torch.Size([3, 7, 8])
torch.Size([3, 3, 8])
torch.Size([3, 7, 9])
torch.Size([3, 3, 9])
torch.Size([2, 7, 10])
torch.Size([2, 3, 10])
torch.Size([2, 7, 11])
torch.Size([2, 3, 11])
torch.Size([5, 7, 12])
torch.Size([5, 3, 12])
torch.Size([6, 7, 13])
torch.Size([6, 3, 13])
torch.Size([5, 7, 14])
torch.Size([5, 3, 14])
torch.Size([4, 7, 15])
torch.Size([4, 3, 15])
torch.Size([1, 7, 16])
torch.Size([1, 3, 16])
torch.Size([1, 7, 17])
torch.Size([1, 3, 17])
torch.Size([1, 7, 21])
torch.Size([1, 3, 21])
torch.Size([3, 7, 22])
torch.Size([3, 3, 22])
torch.Size([5, 7, 23])
torch.Size([5, 3, 23])
torch.Size([3, 7, 24])
torch.Size([3, 3, 24])
torch.Size([2, 7, 25])
torch.Size([2, 3, 25])
torch.Size([1, 7, 26])
torch.Size([1, 3, 26])
torch.Size([1, 7, 27])
torch.Size([1, 3, 27])
torch.Size([1, 7, 28])
torch.Size([1, 3, 28])


In [28]:
mlm_tokenizer.batch_decode(hypotheses.reshape(-1, hypotheses.shape[-1]))

["<s>And me trying to cum. I can hear him having to come, but I didn't stop him. Thank God he had the condom on me.</s><pad><pad><pad><pad><pad><pad><pad>",
 "<s>Watch me trying to cum. I just hear it trying to come, but I didn't stop him. Thank God he had a hand on me.</s><pad><pad><pad><pad><pad><pad><pad>",
 "<s>See me trying to cum. I could hear his try to come, but I didn't stop him. Thank God he had that hold on me.</s><pad><pad><pad><pad><pad><pad><pad>",
 '<s> over like, and he was like:</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>',
 '<s> over like that and he was so:</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>',
 '<s> over like this and he was like.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

In [60]:
top_beams

tensor([[11, 10,  9],
        [ 2,  1,  0]], device='cuda:0')

In [24]:
tmp_hypotheses.shape

torch.Size([8, 30, 39])

In [25]:
hypotheses.shape

torch.Size([10, 3, 39])

In [27]:
i=1

In [37]:
expanded_tmp_hypotheses.reshape(-1, expanded_tmp_hypotheses.shape[-1]).shape

torch.Size([240, 39])

In [35]:
tmp_hypotheses.shape

torch.Size([240, 2])

In [11]:
predicted_token_ids.indices.shape

torch.Size([68, 10])

In [13]:
indices_in_mlm_tokens

(tensor([0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4,
         4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7,
         7, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9],
        device='cuda:0'),
 tensor([ 1,  2, 10, 11, 28, 29, 30,  1,  2,  1, 11, 12, 29, 30, 32,  1,  2, 11,
         12, 21, 22, 23, 21, 29, 30, 31, 32, 33,  1,  2, 11, 12, 13, 14,  1,  9,
         11, 12, 13, 21, 22,  1,  2,  8,  9, 10, 20, 21, 22,  1, 12, 13, 20, 21,
          1,  2,  3,  4,  5,  6,  9, 10, 11, 12, 13, 14, 15, 16],
        device='cuda:0'))

In [124]:
## 일단 무조건 한 스텝 extend하고, 그 다음에 extend한 곳의 값들을 수정하는 방식으로 해보는건 어떨까? 
hypotheses = torch.cat([hypotheses, masked_sequence[:, None, i].unsqueeze(1).expand(-1, config['beam_size'], 1)],dim=-1)

In [126]:
skipping_batch_ids = (masked_sequence[:, i] == mlm_tokenizer.mask_token_id).nonzero()

In [127]:
skipping_batch_ids ## for loop over these indices?

tensor([[0],
        [1],
        [2],
        [3],
        [5],
        [6],
        [7],
        [8],
        [9]], device='cuda:0')

In [61]:
hypotheses[skipping_batch_ids.squeeze()]

tensor([], device='cuda:0', size=(10, 3, 0), dtype=torch.int64)

In [63]:
masked_sequence[skipping_batch_ids.squeeze(), i]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')

In [70]:
masked_sequence[skipping_batch_ids.squeeze(), i].unsqueeze(1).unsqueeze(1).expand(len(skipping_batch_ids), config['beam_size'], 1).shape

torch.Size([10, 3, 1])

In [41]:
masked_sequence[skipping_batch_ids, i].shape

torch.Size([1, 1])

In [ ]:
hypotheses = [torch.LongTensor([]).to(config['device'])]
L = masked_sequence.size(-1)
for i in range(L):
    

In [14]:
hypotheses = [torch.LongTensor([]).to(config['device'])]
L = masked_sequence.size(-1)

for i in range(L):
    if masked_sequence[:, i] != mlm_tokenizer.mask_token_id:
        hypotheses = list(torch.cat([torch.stack(hypotheses,dim=0), 
                                    masked_sequence[:, i].unsqueeze(0).repeat((len(hypotheses),1)).to(config['device'])],dim=-1))
    else:
        num_hypotheses = len(hypotheses)
        hypotheses = torch.stack(hypotheses,dim=0).unsqueeze(0)
        hypotheses = hypotheses.repeat(config['k_per_location'], 1, 1)
        candidates = predicted_token_ids.indices[torch.where(indices_in_mlm_tokens == i)[0], :].to(config['device']).T.unsqueeze(1) ## 이부분이 바뀌어야 함
        candidates = candidates.repeat(1, num_hypotheses, 1)
        hypotheses_exp = torch.cat([hypotheses, candidates], dim=-1)
        hypotheses_exp = hypotheses_exp.view(-1, hypotheses_exp.shape[-1])
        hypotheses_exp = list(hypotheses_exp)

        losses = []
        loss_weights = [1 - config['closs_weight'], config['closs_weight']]
        for hyp in hypotheses_exp:
            curr_loss = 0.0
            for lossid, lossname in enumerate(config["losses"]):
                with torch.no_grad():
                    lossvalue = lossfns[lossid].compute_gold_loss(
                        source_text, mlm_tokenizer.decode(hyp, skip_special_tokens=True),
                        label_id=config['target_label_ids'][lossid],
                    )
                curr_loss += loss_weights[lossid] * lossvalue.item()
            losses.append(curr_loss)

        hypotheses = sorted(zip(hypotheses_exp, losses), key=lambda x: x[1])[:config['beam_size']]
        hypotheses = [x[0] for x in hypotheses]

TypeError: where(): argument 'condition' (position 1) must be Tensor, not bool

In [ ]:

def beam_rerank_v0(source_text, ## text (too arbitrary?)
                    masked_sequence, ## in mlm tokenizer's tokens
                    indices_in_mlm_tokens,
                    predicted_token_ids,
                    mlm_tokenizer, 
                    lossfns,
                    config, 
                    beam_size = 5):
    
    hypotheses = [torch.LongTensor([]).to(config['device'])]
    L = masked_sequence.size(-1)

    for i in range(L):
        if masked_sequence[0, i] != mlm_tokenizer.mask_token_id:
            hypotheses = list(torch.cat([torch.stack(hypotheses,dim=0), 
                                        masked_sequence[:, i].unsqueeze(0).repeat((len(hypotheses),1)).to(config['device'])],dim=-1))
        else:
            num_hypotheses = len(hypotheses)
            hypotheses = torch.stack(hypotheses,dim=0).unsqueeze(0)
            hypotheses = hypotheses.repeat(config['k_per_location'], 1, 1)
            candidates = predicted_token_ids.indices[torch.where(indices_in_mlm_tokens == i)[0], :].to(config['device']).T.unsqueeze(1)
            candidates = candidates.repeat(1, num_hypotheses, 1)
            hypotheses_exp = torch.cat([hypotheses, candidates], dim=-1)
            hypotheses_exp = hypotheses_exp.view(-1, hypotheses_exp.shape[-1])
            hypotheses_exp = list(hypotheses_exp)

            losses = []
            loss_weights = [1 - wandb.config.closs_weight, wandb.config.closs_weight]
            for hyp in hypotheses_exp:
                curr_loss = 0.0
                for lossid, lossname in enumerate(config["losses"]):
                    with torch.no_grad():
                        lossvalue = lossfns[lossid].compute_gold_loss(
                            source_text, mlm_tokenizer.decode(hyp, skip_special_tokens=True),
                            label_id=config['target_label_ids'][lossid],
                        )
                    curr_loss += loss_weights[lossid] * lossvalue.item()
                losses.append(curr_loss)

            hypotheses = sorted(zip(hypotheses_exp, losses), key=lambda x: x[1])[:beam_size]
            hypotheses = [x[0] for x in hypotheses]
            
    return [mlm_tokenizer.decode(x, skip_special_tokens=True) for x in hypotheses]
